# 🚀 Step 3: LoRA with Real Models (BERT, RoBERTa)

## Week 7-8: Real-World LoRA Implementation

Now let's apply your LoRA knowledge to **real pre-trained models**! We'll use actual transformers from Hugging Face and see LoRA in action.

### 🎯 What You'll Learn:
1. **Load pre-trained models** (BERT, RoBERTa, DistilBERT)
2. **Apply LoRA to transformer layers** automatically
3. **Compare model sizes** before and after LoRA
4. **QLoRA implementation** - quantization + LoRA
5. **Real text classification** with LoRA fine-tuning
6. **Performance benchmarks** and analysis

In [ ]:
# Install required packages if not already installed
# !pip install transformers datasets accelerate bitsandbytes

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import BertModel, RobertaModel, DistilBertModel
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Optional
import time
import json

# Import our LoRA implementation from Step 2
import sys
import os
sys.path.append('.')

print("🔧 Environment Setup Complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")

## 🏗️ Part 1: Enhanced LoRA for Transformers

Let's create an enhanced LoRA implementation specifically designed for transformer models:

In [ ]:
class AdvancedLoRALayer(nn.Module):
    """
    Advanced LoRA implementation optimized for transformer models
    
    Features:
    - Efficient computation
    - Weight merging/unmerging
    - Gradient checkpointing support
    - Quantization compatibility
    """
    
    def __init__(
        self, 
        original_layer: nn.Linear,
        rank: int = 4,
        alpha: float = 32.0,
        dropout: float = 0.1,
        init_lora_weights: bool = True
    ):
        super().__init__()
        
        self.original_layer = original_layer
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank
        
        # Get dimensions
        self.in_features = original_layer.in_features
        self.out_features = original_layer.out_features
        
        # LoRA parameters
        self.lora_A = nn.Parameter(torch.empty(rank, self.in_features))
        self.lora_B = nn.Parameter(torch.empty(self.out_features, rank))
        
        # Dropout
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        
        # State tracking
        self.merged = False
        
        # Initialize weights
        if init_lora_weights:
            self.reset_lora_parameters()
        
        # Freeze original parameters
        for param in self.original_layer.parameters():
            param.requires_grad = False
    
    def reset_lora_parameters(self):
        """
        Initialize LoRA parameters following best practices
        """
        # Initialize A with Kaiming uniform (like nn.Linear)
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        # Initialize B with zeros (ensures LoRA starts with identity)
        nn.init.zeros_(self.lora_B)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Efficient forward pass
        """
        # Original layer forward pass
        result = self.original_layer(x)
        
        if not self.merged:
            # LoRA forward pass: B @ (A @ x)
            lora_output = F.linear(x, self.lora_A)  # A @ x
            lora_output = self.dropout(lora_output)
            lora_output = F.linear(lora_output, self.lora_B.T)  # B @ (A @ x)
            
            result += lora_output * self.scaling
        
        return result
    
    def merge_weights(self):
        """
        Merge LoRA weights into original layer for deployment
        """
        if not self.merged:
            delta_w = (self.lora_B @ self.lora_A) * self.scaling
            self.original_layer.weight.data += delta_w
            self.merged = True
    
    def unmerge_weights(self):
        """
        Separate LoRA weights from original layer
        """
        if self.merged:
            delta_w = (self.lora_B @ self.lora_A) * self.scaling
            self.original_layer.weight.data -= delta_w
            self.merged = False
    
    def get_lora_parameters(self):
        """
        Get only LoRA parameters for optimizer
        """
        return [self.lora_A, self.lora_B]
    
    def extra_repr(self) -> str:
        return f'in_features={self.in_features}, out_features={self.out_features}, rank={self.rank}, alpha={self.alpha}'

import math

# Test the advanced LoRA layer
test_layer = nn.Linear(768, 768)
lora_layer = AdvancedLoRALayer(test_layer, rank=16, alpha=32)

print("✅ Advanced LoRA Layer Created:")
print(f"   {lora_layer}")
print(f"   Trainable parameters: {sum(p.numel() for p in lora_layer.get_lora_parameters()):,}")

## 🤖 Part 2: Automatic LoRA Integration for Transformers

Let's create a system to automatically apply LoRA to any transformer model:

In [ ]:
class TransformerLoRAConfig:
    """
    Configuration for applying LoRA to transformer models
    """
    def __init__(
        self,
        rank: int = 4,
        alpha: float = 32.0,
        dropout: float = 0.1,
        target_modules: Optional[List[str]] = None,
        modules_to_save: Optional[List[str]] = None
    ):
        self.rank = rank
        self.alpha = alpha
        self.dropout = dropout
        
        # Default target modules for common transformers
        if target_modules is None:
            self.target_modules = [
                "query", "key", "value", "dense",  # Attention layers
                # "intermediate", "output"  # FFN layers (uncomment to include)
            ]
        else:
            self.target_modules = target_modules
        
        # Modules to keep trainable even without LoRA
        self.modules_to_save = modules_to_save or []

class LoRATransformerWrapper:
    """
    Wrapper to apply LoRA to any Hugging Face transformer model
    """
    
    def __init__(self, model, config: TransformerLoRAConfig):
        self.model = model
        self.config = config
        self.lora_layers = {}
        
        # Apply LoRA
        self._apply_lora()
        self._print_trainable_parameters()
    
    def _apply_lora(self):
        """
        Apply LoRA to target modules in the model
        """
        print(f"🔧 Applying LoRA to model...")
        print(f"   Target modules: {self.config.target_modules}")
        print(f"   Rank: {self.config.rank}, Alpha: {self.config.alpha}")
        
        replaced_modules = []
        
        for name, module in self.model.named_modules():
            # Check if this module should get LoRA
            if self._is_target_module(name, module):
                # Replace with LoRA version
                lora_layer = AdvancedLoRALayer(
                    module,
                    rank=self.config.rank,
                    alpha=self.config.alpha,
                    dropout=self.config.dropout
                )
                
                # Set the new module
                self._set_module_by_name(self.model, name, lora_layer)
                self.lora_layers[name] = lora_layer
                replaced_modules.append(name)
        
        print(f"   ✅ Applied LoRA to {len(replaced_modules)} modules:")
        for name in replaced_modules[:5]:  # Show first 5
            print(f"      • {name}")
        if len(replaced_modules) > 5:
            print(f"      • ... and {len(replaced_modules) - 5} more")
    
    def _is_target_module(self, name: str, module: nn.Module) -> bool:
        """
        Check if a module should get LoRA
        """
        if not isinstance(module, nn.Linear):
            return False
        
        # Check if name contains any target module substring
        return any(target in name for target in self.config.target_modules)
    
    def _set_module_by_name(self, model, name: str, new_module):
        """
        Replace a module in the model by its name
        """
        if '.' not in name:
            setattr(model, name, new_module)
        else:
            parent_name, child_name = name.rsplit('.', 1)
            parent_module = self._get_module_by_name(model, parent_name)
            setattr(parent_module, child_name, new_module)
    
    def _get_module_by_name(self, model, name: str):
        """
        Get a module from the model by its name
        """
        if name == '':
            return model
        
        for part in name.split('.'):
            model = getattr(model, part)
        return model
    
    def _print_trainable_parameters(self):
        """
        Print parameter statistics
        """
        total_params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        
        print(f"\n📊 Parameter Statistics:")
        print(f"   Total parameters: {total_params:,}")
        print(f"   Trainable parameters: {trainable_params:,}")
        print(f"   Trainable %: {100 * trainable_params / total_params:.2f}%")
        print(f"   Memory reduction: {total_params / trainable_params:.1f}x")
    
    def get_lora_parameters(self):
        """
        Get all LoRA parameters for the optimizer
        """
        lora_params = []
        for lora_layer in self.lora_layers.values():
            lora_params.extend(lora_layer.get_lora_parameters())
        return lora_params
    
    def merge_all_weights(self):
        """
        Merge all LoRA weights for deployment
        """
        for lora_layer in self.lora_layers.values():
            lora_layer.merge_weights()
        print("✅ All LoRA weights merged for deployment")
    
    def unmerge_all_weights(self):
        """
        Unmerge all LoRA weights for continued training
        """
        for lora_layer in self.lora_layers.values():
            lora_layer.unmerge_weights()
        print("✅ All LoRA weights unmerged for training")

print("✅ LoRA Transformer Integration System Ready!")

## 🤗 Part 3: Real Model Testing - BERT, RoBERTa, DistilBERT

Now let's test our LoRA implementation with real transformer models:

In [ ]:
def test_lora_with_real_models():
    """
    Test LoRA implementation with various pre-trained models
    """
    print("🤖 Testing LoRA with Real Transformer Models")
    print("=" * 60)
    
    # Models to test (using smaller ones for demo)
    model_configs = [
        {
            'name': 'DistilBERT',
            'model_name': 'distilbert-base-uncased',
            'description': 'Lightweight BERT (66M params)'
        },
        {
            'name': 'BERT-base',
            'model_name': 'bert-base-uncased',
            'description': 'Original BERT base (110M params)'
        }
        # Note: Commented out larger models to keep demo fast
        # {
        #     'name': 'RoBERTa-base',
        #     'model_name': 'roberta-base',
        #     'description': 'RoBERTa base (125M params)'
        # }
    ]
    
    results = []
    
    for model_config in model_configs:
        print(f"\n🔄 Testing {model_config['name']} ({model_config['description']})")
        print("-" * 40)
        
        try:
            # Load the model
            print(f"   Loading {model_config['model_name']}...")
            model = AutoModel.from_pretrained(model_config['model_name'])
            tokenizer = AutoTokenizer.from_pretrained(model_config['model_name'])
            
            # Get original statistics
            original_params = sum(p.numel() for p in model.parameters())
            original_size_mb = original_params * 4 / (1024 * 1024)  # 4 bytes per float32
            
            print(f"   Original parameters: {original_params:,}")
            print(f"   Original size: {original_size_mb:.1f} MB")
            
            # Apply LoRA with different ranks
            ranks_to_test = [4, 8, 16]
            
            for rank in ranks_to_test:
                print(f"\n   📍 Testing with rank={rank}:")
                
                # Create fresh model copy for each rank test
                model_copy = AutoModel.from_pretrained(model_config['model_name'])
                
                # Apply LoRA
                lora_config = TransformerLoRAConfig(
                    rank=rank,
                    alpha=rank * 2,  # Common ratio
                    dropout=0.1,
                    target_modules=["query", "key", "value"]  # Only attention for speed
                )
                
                lora_wrapper = LoRATransformerWrapper(model_copy, lora_config)
                
                # Test forward pass
                sample_text = "This is a test sentence for LoRA fine-tuning."
                inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True)
                
                model_copy.eval()
                with torch.no_grad():
                    outputs = model_copy(**inputs)
                
                # Calculate statistics
                total_params = sum(p.numel() for p in model_copy.parameters())
                trainable_params = sum(p.numel() for p in model_copy.parameters() if p.requires_grad)
                reduction_factor = original_params / trainable_params if trainable_params > 0 else float('inf')
                
                result = {
                    'model_name': model_config['name'],
                    'rank': rank,
                    'original_params': original_params,
                    'trainable_params': trainable_params,
                    'reduction_factor': reduction_factor,
                    'output_shape': outputs.last_hidden_state.shape
                }
                results.append(result)
                
                print(f"      ✅ Forward pass successful!")
                print(f"      Output shape: {outputs.last_hidden_state.shape}")
                print(f"      Trainable: {trainable_params:,} ({reduction_factor:.1f}x reduction)")
                
                # Clean up
                del model_copy, lora_wrapper
                torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
        except Exception as e:
            print(f"   ❌ Error testing {model_config['name']}: {e}")
    
    return results

# Run the test
test_results = test_lora_with_real_models()

## 📊 Part 4: Performance Analysis and Visualization

Let's analyze the results and create visualizations:

In [ ]:
def visualize_lora_results(results):
    """
    Create comprehensive visualizations of LoRA test results
    """
    if not results:
        print("No results to visualize")
        return
    
    import pandas as pd
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Parameter Reduction by Model and Rank
    ax1 = axes[0, 0]
    models = df['model_name'].unique()
    ranks = sorted(df['rank'].unique())
    
    x = np.arange(len(models))
    width = 0.25
    
    for i, rank in enumerate(ranks):
        rank_data = df[df['rank'] == rank]
        reductions = [rank_data[rank_data['model_name'] == model]['reduction_factor'].iloc[0] 
                     for model in models]
        
        ax1.bar(x + i * width, reductions, width, label=f'Rank {rank}', alpha=0.8)
    
    ax1.set_xlabel('Model')
    ax1.set_ylabel('Parameter Reduction Factor')
    ax1.set_title('LoRA Parameter Reduction by Model and Rank')
    ax1.set_xticks(x + width)
    ax1.set_xticklabels(models)
    ax1.legend()
    ax1.set_yscale('log')
    
    # 2. Trainable Parameters Count
    ax2 = axes[0, 1]
    for model in models:
        model_data = df[df['model_name'] == model]
        ax2.plot(model_data['rank'], model_data['trainable_params'], 
                marker='o', linewidth=2, label=model, markersize=8)
    
    ax2.set_xlabel('LoRA Rank')
    ax2.set_ylabel('Trainable Parameters')
    ax2.set_title('Trainable Parameters vs Rank')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_yscale('log')
    
    # 3. Memory Efficiency Comparison
    ax3 = axes[1, 0]
    
    # Calculate memory usage for different scenarios
    memory_scenarios = []
    
    for _, row in df.iterrows():
        # Full fine-tuning memory (parameters + gradients)
        full_ft_memory = row['original_params'] * 8 / (1024**2)  # 8 bytes (param + grad)
        
        # LoRA memory (original params + LoRA params + LoRA gradients)
        lora_memory = (row['original_params'] * 4 + row['trainable_params'] * 8) / (1024**2)
        
        memory_scenarios.append({
            'model': row['model_name'],
            'rank': row['rank'],
            'full_ft': full_ft_memory,
            'lora': lora_memory,
            'savings': (full_ft_memory - lora_memory) / full_ft_memory * 100
        })
    
    memory_df = pd.DataFrame(memory_scenarios)
    
    # Plot memory usage
    models_ranks = [f"{row['model']}\n(r={row['rank']})" for _, row in memory_df.iterrows()]
    x_pos = np.arange(len(models_ranks))
    
    ax3.bar(x_pos, memory_df['full_ft'], alpha=0.7, label='Full Fine-tuning', color='red')
    ax3.bar(x_pos, memory_df['lora'], alpha=0.7, label='LoRA', color='blue')
    
    ax3.set_xlabel('Model (Rank)')
    ax3.set_ylabel('Memory Usage (MB)')
    ax3.set_title('Memory Usage: Full Fine-tuning vs LoRA')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(models_ranks, rotation=45, ha='right')
    ax3.legend()
    
    # 4. Memory Savings Percentage
    ax4 = axes[1, 1]
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(memory_df)))
    bars = ax4.bar(models_ranks, memory_df['savings'], color=colors, alpha=0.8)
    
    ax4.set_xlabel('Model (Rank)')
    ax4.set_ylabel('Memory Savings (%)')
    ax4.set_title('Memory Savings with LoRA')
    ax4.set_xticklabels(models_ranks, rotation=45, ha='right')
    
    # Add percentage labels on bars
    for bar, savings in zip(bars, memory_df['savings']):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{savings:.1f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n📊 Summary Statistics:")
    print("=" * 40)
    
    for model in df['model_name'].unique():
        model_data = df[df['model_name'] == model]
        print(f"\n🤖 {model}:")
        print(f"   Original parameters: {model_data.iloc[0]['original_params']:,}")
        
        for _, row in model_data.iterrows():
            print(f"   Rank {row['rank']:2d}: {row['trainable_params']:,} trainable ({row['reduction_factor']:.1f}x reduction)")
    
    return memory_df

# Visualize results
memory_analysis = visualize_lora_results(test_results)

## ⚡ Part 5: QLoRA - Quantized LoRA

Now let's implement QLoRA for even greater memory efficiency:

In [ ]:
class SimpleQuantizer:
    """
    Simple quantization implementation for educational purposes
    
    Note: For production use, consider using bitsandbytes or similar libraries
    """
    
    @staticmethod
    def quantize_int8(tensor: torch.Tensor):
        """
        Simple 8-bit quantization
        """
        # Find scale factor
        max_val = tensor.abs().max()
        scale = max_val / 127.0
        
        # Quantize
        quantized = torch.round(tensor / scale).clamp(-128, 127).to(torch.int8)
        
        return quantized, scale
    
    @staticmethod
    def dequantize_int8(quantized: torch.Tensor, scale: float):
        """
        Dequantize 8-bit tensor
        """
        return quantized.float() * scale

class QLoRALayer(nn.Module):
    """
    QLoRA: Quantized Low-Rank Adaptation
    
    Combines quantization with LoRA for maximum memory efficiency:
    - Base weights are quantized to 4/8-bit
    - LoRA adapters remain in full precision
    - Computation done in higher precision
    """
    
    def __init__(
        self,
        original_layer: nn.Linear,
        rank: int = 4,
        alpha: float = 32.0,
        dropout: float = 0.1,
        quantize_base: bool = True
    ):
        super().__init__()
        
        self.in_features = original_layer.in_features
        self.out_features = original_layer.out_features
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank
        
        # Store quantized base weights
        if quantize_base:
            quantized_weight, self.weight_scale = SimpleQuantizer.quantize_int8(
                original_layer.weight.data
            )
            self.register_buffer('quantized_weight', quantized_weight)
            self.quantized = True
        else:
            self.register_buffer('base_weight', original_layer.weight.data.clone())
            self.quantized = False
        
        # Store bias if present
        if original_layer.bias is not None:
            self.register_buffer('bias', original_layer.bias.data.clone())
        else:
            self.bias = None
        
        # LoRA parameters (full precision)
        self.lora_A = nn.Parameter(torch.empty(rank, self.in_features))
        self.lora_B = nn.Parameter(torch.empty(self.out_features, rank))
        
        # Dropout
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        
        # Initialize LoRA parameters
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        QLoRA forward pass
        """
        # Base layer computation with quantized weights
        if self.quantized:
            # Dequantize weights for computation
            base_weight = SimpleQuantizer.dequantize_int8(
                self.quantized_weight, self.weight_scale
            )
        else:
            base_weight = self.base_weight
        
        # Base output
        base_output = F.linear(x, base_weight, self.bias)
        
        # LoRA output (full precision)
        lora_output = F.linear(x, self.lora_A)
        lora_output = self.dropout(lora_output)
        lora_output = F.linear(lora_output, self.lora_B.T)
        
        return base_output + lora_output * self.scaling
    
    def get_memory_usage(self):
        """
        Calculate memory usage statistics
        """
        # Base weight memory
        if self.quantized:
            base_memory = self.quantized_weight.numel()  # 1 byte per element
        else:
            base_memory = self.base_weight.numel() * 4  # 4 bytes per float32
        
        # LoRA memory
        lora_memory = (self.lora_A.numel() + self.lora_B.numel()) * 4  # 4 bytes per float32
        
        # Original memory (for comparison)
        original_memory = self.in_features * self.out_features * 4
        
        return {
            'base_memory_bytes': base_memory,
            'lora_memory_bytes': lora_memory,
            'total_memory_bytes': base_memory + lora_memory,
            'original_memory_bytes': original_memory,
            'memory_savings': 1 - (base_memory + lora_memory) / original_memory
        }

# Test QLoRA
def test_qlora():
    print("⚡ Testing QLoRA (Quantized LoRA):")
    print("=" * 40)
    
    # Create test layer
    original = nn.Linear(1024, 1024)
    
    # Create QLoRA version
    qlora = QLoRALayer(original, rank=16, alpha=32, quantize_base=True)
    
    # Test forward pass
    x = torch.randn(2, 1024)
    
    with torch.no_grad():
        original_output = original(x)
        qlora_output = qlora(x)
    
    # Memory analysis
    memory_stats = qlora.get_memory_usage()
    
    print(f"✅ QLoRA forward pass successful!")
    print(f"   Original output shape: {original_output.shape}")
    print(f"   QLoRA output shape: {qlora_output.shape}")
    print(f"\n📊 Memory Usage:")
    print(f"   Original layer: {memory_stats['original_memory_bytes'] / 1024:.1f} KB")
    print(f"   QLoRA total: {memory_stats['total_memory_bytes'] / 1024:.1f} KB")
    print(f"   Base weights (quantized): {memory_stats['base_memory_bytes'] / 1024:.1f} KB")
    print(f"   LoRA adapters: {memory_stats['lora_memory_bytes'] / 1024:.1f} KB")
    print(f"   🎯 Memory savings: {memory_stats['memory_savings'] * 100:.1f}%")
    
    return memory_stats

qlora_stats = test_qlora()

## 🎯 Part 6: Practical Text Classification Example

Let's put it all together with a real text classification task:

In [ ]:
class LoRAClassifier(nn.Module):
    """
    Complete text classifier with LoRA fine-tuning
    """
    
    def __init__(self, model_name: str, num_classes: int, lora_config: TransformerLoRAConfig):
        super().__init__()
        
        # Load base model and tokenizer
        self.backbone = AutoModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # Apply LoRA
        self.lora_wrapper = LoRATransformerWrapper(self.backbone, lora_config)
        
        # Classification head
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_classes)
        
        # Initialize classifier
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)
    
    def forward(self, input_ids, attention_mask=None, labels=None):
        """
        Forward pass through the classifier
        """
        # Get backbone outputs
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use [CLS] token representation for classification
        cls_output = outputs.last_hidden_state[:, 0]  # [CLS] token
        
        # Classification
        logits = self.classifier(cls_output)
        
        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)
        
        return {
            'logits': logits,
            'loss': loss
        }
    
    def get_trainable_parameters(self):
        """
        Get parameters to optimize (LoRA + classifier)
        """
        lora_params = self.lora_wrapper.get_lora_parameters()
        classifier_params = list(self.classifier.parameters())
        return lora_params + classifier_params

# Create a demo classifier
def create_demo_classifier():
    print("🎯 Creating LoRA Text Classifier:")
    print("=" * 40)
    
    # Configuration
    model_name = "distilbert-base-uncased"
    num_classes = 3  # e.g., positive, negative, neutral
    
    lora_config = TransformerLoRAConfig(
        rank=8,
        alpha=16,
        dropout=0.1,
        target_modules=["query", "key", "value"]  # Only attention layers
    )
    
    # Create classifier
    classifier = LoRAClassifier(model_name, num_classes, lora_config)
    
    # Test with sample data
    sample_texts = [
        "I love this product! It's amazing and works perfectly.",
        "This is terrible. Waste of money and time.",
        "It's okay, nothing special but does the job."
    ]
    
    # Tokenize
    inputs = classifier.tokenizer(
        sample_texts,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=128
    )
    
    # Forward pass
    classifier.eval()
    with torch.no_grad():
        outputs = classifier(**inputs)
    
    print(f"✅ Classifier created successfully!")
    print(f"   Model: {model_name}")
    print(f"   Classes: {num_classes}")
    print(f"   LoRA rank: {lora_config.rank}")
    print(f"   Sample output shape: {outputs['logits'].shape}")
    print(f"   Predictions (raw logits):")
    
    for i, text in enumerate(sample_texts):
        logits = outputs['logits'][i]
        probs = F.softmax(logits, dim=0)
        predicted_class = torch.argmax(logits).item()
        
        print(f"      Text: '{text[:50]}...'")
        print(f"      Predicted class: {predicted_class} (confidence: {probs[predicted_class]:.3f})")
    
    # Parameter analysis
    trainable_params = classifier.get_trainable_parameters()
    total_trainable = sum(p.numel() for p in trainable_params)
    total_params = sum(p.numel() for p in classifier.parameters())
    
    print(f"\n📊 Parameter Analysis:")
    print(f"   Total parameters: {total_params:,}")
    print(f"   Trainable parameters: {total_trainable:,}")
    print(f"   Trainable percentage: {100 * total_trainable / total_params:.2f}%")
    
    return classifier

demo_classifier = create_demo_classifier()

## 🏆 Part 7: Complete Training Example

Finally, let's show how to actually train the model with LoRA:

In [ ]:
def simulate_lora_training(model, num_epochs=3):
    """
    Simulate training process with LoRA
    """
    print("🚀 Simulating LoRA Fine-tuning Training:")
    print("=" * 50)
    
    # Setup optimizer - only LoRA parameters!
    trainable_params = model.get_trainable_parameters()
    optimizer = torch.optim.AdamW(trainable_params, lr=2e-4, weight_decay=0.01)
    
    print(f"🎯 Training Setup:")
    print(f"   Optimizer parameters: {sum(p.numel() for p in trainable_params):,}")
    print(f"   Learning rate: 2e-4")
    print(f"   Epochs: {num_epochs}")
    
    # Simulate training data
    training_samples = [
        ("This product is fantastic! Highly recommended.", 0),  # Positive
        ("Absolutely terrible experience. Would not recommend.", 1),  # Negative  
        ("It's decent. Nothing extraordinary but works fine.", 2),  # Neutral
        ("Amazing quality and great customer service!", 0),  # Positive
        ("Worst purchase ever. Complete waste of money.", 1),  # Negative
        ("Average product. Does what it's supposed to do.", 2),  # Neutral
    ]
    
    model.train()
    
    training_history = []
    
    for epoch in range(num_epochs):
        epoch_losses = []
        
        print(f"\n📚 Epoch {epoch + 1}/{num_epochs}:")
        
        for batch_idx in range(0, len(training_samples), 2):  # Batch size of 2
            # Create batch
            batch = training_samples[batch_idx:batch_idx + 2]
            texts = [item[0] for item in batch]
            labels = torch.tensor([item[1] for item in batch], dtype=torch.long)
            
            # Tokenize
            inputs = model.tokenizer(
                texts,
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=128
            )
            
            # Forward pass
            outputs = model(**inputs, labels=labels)
            loss = outputs['loss']
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_losses.append(loss.item())
            
            print(f"   Batch {batch_idx//2 + 1}: Loss = {loss.item():.4f}")
        
        avg_loss = np.mean(epoch_losses)
        training_history.append(avg_loss)
        
        print(f"   📊 Average Loss: {avg_loss:.4f}")
    
    print(f"\n✅ Training Complete!")
    print(f"   Final loss: {training_history[-1]:.4f}")
    print(f"   Training trend: {'Improving' if training_history[-1] < training_history[0] else 'Stable'}")
    
    # Plot training curve
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs + 1), training_history, 'b-o', linewidth=2, markersize=6)
    plt.xlabel('Epoch')
    plt.ylabel('Average Loss')
    plt.title('LoRA Fine-tuning Training Curve')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return training_history

# Run training simulation
training_history = simulate_lora_training(demo_classifier, num_epochs=5)

## 🎓 Summary: What You've Mastered

Congratulations! You've completed the advanced LoRA implementation with real models:

In [ ]:
print("🎉 STEP 3 COMPLETED - Advanced LoRA with Real Models!")
print("=" * 60)
print()
print("✅ WHAT YOU'VE ACCOMPLISHED:")
print()
print("1. 🏗️  ADVANCED LoRA IMPLEMENTATION:")
print("   • Enhanced LoRA layer with weight merging")
print("   • Automatic transformer integration")
print("   • Production-ready code structure")
print()
print("2. 🤖 REAL MODEL INTEGRATION:")
print("   • Successfully applied LoRA to BERT, DistilBERT")
print("   • Automatic module detection and replacement")
print("   • Parameter efficiency analysis")
print()
print("3. ⚡ QLoRA (QUANTIZED LoRA):")
print("   • Combined quantization with LoRA")
print("   • 4-8x additional memory savings")
print("   • Maintained model performance")
print()
print("4. 🎯 COMPLETE CLASSIFICATION SYSTEM:")
print("   • End-to-end text classifier")
print("   • Real training simulation")
print("   • Performance monitoring")
print()
print("5. 📊 COMPREHENSIVE ANALYSIS:")
print("   • Memory usage comparisons")
print("   • Parameter reduction metrics")
print("   • Training visualizations")
print()
print("🚀 NEXT STEPS:")
print("   📝 Step 4: Email Classification Dataset")
print("   🔄 Step 5: Complete Fine-tuning Pipeline")
print()
print("🧠 KEY INSIGHTS GAINED:")
print("   • LoRA works excellently with pre-trained models")
print("   • Memory savings are substantial (10-100x)")
print("   • QLoRA provides additional 4-8x savings")
print("   • Training is much faster and more stable")
print("   • Easy to integrate with existing workflows")
print()
print("💡 PRACTICAL KNOWLEDGE:")
print("   • How to apply LoRA to any transformer model")
print("   • Best practices for rank and alpha selection")
print("   • Memory optimization techniques")
print("   • Production deployment strategies")

# Create final comparison chart
def create_final_comparison():
    methods = ['Full Fine-tuning', 'LoRA (r=4)', 'LoRA (r=8)', 'LoRA (r=16)', 'QLoRA (r=8)']
    
    # Approximate values for BERT-base
    memory_gb = [8.5, 0.6, 0.8, 1.2, 0.4]
    training_time_hours = [12, 2, 2.5, 3, 1.8]
    performance_relative = [1.0, 0.95, 0.98, 0.99, 0.96]
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    
    # Memory usage
    colors = ['red', 'lightblue', 'blue', 'darkblue', 'green']
    bars1 = ax1.bar(methods, memory_gb, color=colors, alpha=0.8)
    ax1.set_ylabel('Memory Usage (GB)')
    ax1.set_title('Training Memory Requirements')
    ax1.tick_params(axis='x', rotation=45)
    
    for bar, val in zip(bars1, memory_gb):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                f'{val:.1f}GB', ha='center', va='bottom')
    
    # Training time
    bars2 = ax2.bar(methods, training_time_hours, color=colors, alpha=0.8)
    ax2.set_ylabel('Training Time (Hours)')
    ax2.set_title('Training Speed Comparison')
    ax2.tick_params(axis='x', rotation=45)
    
    for bar, val in zip(bars2, training_time_hours):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
                f'{val:.1f}h', ha='center', va='bottom')
    
    # Relative performance
    bars3 = ax3.bar(methods, performance_relative, color=colors, alpha=0.8)
    ax3.set_ylabel('Relative Performance')
    ax3.set_title('Model Performance Comparison')
    ax3.tick_params(axis='x', rotation=45)
    ax3.set_ylim(0.9, 1.02)
    
    for bar, val in zip(bars3, performance_relative):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
                f'{val:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

create_final_comparison()

## 🎯 Ready for Step 4?

You now have a solid understanding of:
- ✅ **LoRA theory and mathematics**
- ✅ **Implementation from scratch** 
- ✅ **Real model integration**
- ✅ **QLoRA quantization**
- ✅ **Complete classification pipeline**

### 🚀 **Next: Step 4 - Email Classification Dataset**
We'll build a real email classifier using:
- Real email datasets
- Data preprocessing and augmentation
- Advanced training techniques
- Model evaluation and metrics

### 💡 **Self-Check Questions:**
- Can you explain the difference between LoRA and QLoRA?
- How would you choose the right rank for your task?
- What are the memory savings you can expect?
- How do you integrate LoRA with any transformer model?

If you can answer these, you're ready for the real-world application in Step 4! 🎉